In [1]:
import pandas as pd

READ THE FILE FIRST

In [2]:
def read_imdb_data_with_pandas(file_path):
    # Load the CSV file into a Pandas DataFrame
    data = pd.read_csv(file_path)
    
    # Remove any extra spaces around the '|'
    # This will ensure that all actor names are consistent and don't have extra spaces
    data['Actors'] = data['Actors'].apply(lambda x: '|'.join([actor.strip() for actor in x.split('|')]))
    
    return data

data = read_imdb_data_with_pandas("IMDB-Movie-Data.csv")
#data.head(10)
data["Actors"].head(10)

0    Christian Bale|Heath Ledger|Aaron Eckhart|Mich...
1    Leonardo DiCaprio|Joseph Gordon-Levitt|Ellen P...
2    Aamir Khan|Sakshi Tanwar|Fatima Sana Shaikh|Sa...
3    Matthew McConaughey|Anne Hathaway|Jessica Chas...
4    Ryunosuke Kamiki|Mone Kamishiraishi|Ryo Narita...
5    Francois Cluzet|Omar Sy|Anne Le Ny|Audrey Fleurot
6    Christian Bale|Hugh Jackman|Scarlett Johansson...
7    Leonardo DiCaprio|Matt Damon|Jack Nicholson|Ma...
8    Christian Bale|Tom Hardy|Anne Hathaway|Gary Ol...
9    Miles Teller|J.K. Simmons|Melissa Benoist|Paul...
Name: Actors, dtype: object

QUESTION 1

In [3]:
# Question 1
def top_3_movies_with_highest_ratings_in_2016(data):
    # Filter out the movies that is released in 2016
    # Pandas boolean indexing 
    # Use for filtering the DataFrame and data 
    movies_in_2016 = data[data["Year"] == 2016 ]
    # Find the top 3 unique ratings
    top_3_ratings = movies_in_2016["Rating"].drop_duplicates().nlargest(3)
    # Select only the "Title" "Rating" columns from the filtered result
    # Double Bracketes are for multiple columns
    # Filter the movies that have a rating in the top 3 ratings
    answer = movies_in_2016[movies_in_2016["Rating"].isin(top_3_ratings)][["Title", "Rating"]]
    # Sort the answer by rating in descending order
    answer = answer.sort_values(by="Rating", ascending=False)

    return answer

question_1 = top_3_movies_with_highest_ratings_in_2016(data)
question_1

,Title,Rating
2,Dangal,8.8
4,Kimi no na wa,8.6
15,Koe no katachi,8.4


QUESTION 2

In [5]:
# Question 2
def actor_with_highest_average_revenue_per_movie(data):
    # Split the 'Actors' column into individual actors and explode into rows
    actors_data = data.assign(Actors=data["Actors"].str.split('|')).explode("Actors")
    # Group by actor and calc. the average revenue per movie
    actor_revenue = actors_data.groupby("Actors")["Revenue (Millions)"].mean()
    # Find the highest average revenue
    max_average_revenue = actor_revenue.max()
    # Find all actors who have this max. average revenue
    answer = actor_revenue[actor_revenue == max_average_revenue]
    
    return answer

question_2 = actor_with_highest_average_revenue_per_movie(data)
question_2

Actors
Daisy Ridley    936.63
John Boyega     936.63
Name: Revenue (Millions), dtype: float64

QUESTION 3

In [7]:
def average_rating_of_EmmaWatson_movies(data):
    # Filter rows where the "Actors" column contains "Emma Watson"
    emma_watson_movies = data[data["Actors"].str.contains("Emma Watson")]
    # Calc. the mean of the rating in "Rating" column for those movies
    answer = emma_watson_movies["Rating"].mean()

    return answer

question_3 = average_rating_of_EmmaWatson_movies(data)
print(f"The average rating of Emma Watson's movies is {question_3:.3f}")

The average rating of Emma Watson's movies is 7.175


QUESTION 4

In [8]:
def top_3_directors_collaborate_with_most_actors(data):
    # Split "Actors" and explode the list of actors into individual rows
    directors_data = data.assign(Actors=data["Actors"].str.split('|')).explode("Actors")
    # Group by 'Director' and count the number of collaborated actors 
    # nunique(): Count the number of unique actors 
    director_actor_count = directors_data.groupby("Director")["Actors"].nunique()
    # Get the top 3 unique numbers of collaborated actors
    top_3_numbers_collaborated_actors = director_actor_count.drop_duplicates().nlargest(3)
    answer = director_actor_count[director_actor_count.isin(top_3_numbers_collaborated_actors)].reset_index(name="Number of Collaborated Actors")
    # Sort the answer by rating in descending order
    answer = answer.sort_values(by="Number of Collaborated Actors", ascending=False)

    return answer

question_4 = top_3_directors_collaborate_with_most_actors(data)
question_4

,Director,Number of Collaborated Actors
3,Ridley Scott,28
1,M. Night Shyamalan,24
0,Danny Boyle,20
2,Paul W.S. Anderson,20


QUESTION 5

In [9]:
def top_2_actors_playing_most_genres_movies(data):
    # Split 'Actors' and 'Genre' into lists
    actors_genres = data.assign(Actors=data["Actors"].str.split('|'), Genre=data["Genre"].str.split('|'))
    # Explode the 'Actors' column first, then the 'Genre' column
    actors_genres = actors_genres.explode("Actors").explode("Genre")    
    # Group by actor and count the number of unique genres they have played in
    actor_genre_count = actors_genres.groupby("Actors")["Genre"].nunique()
    # Get the top 2 actors with the most unique genres
    top_2_numbers_genres = actor_genre_count.drop_duplicates().nlargest(2)
    answer = actor_genre_count[actor_genre_count.isin(top_2_numbers_genres)].reset_index(name="Number of Genre Movies")
    # Sort the answer by rating in descending order
    answer = answer.sort_values(by="Number of Genre Movies", ascending=False)

    return answer

question_5 = top_2_actors_playing_most_genres_movies(data)
question_5

,Actors,Number of Genre Movies
1,Brad Pitt,14
0,Amy Adams,13
2,Chloe Grace Moretz,13
3,Hugh Jackman,13
4,Johnny Depp,13
5,Scarlett Johansson,13


QUESTION 6

In [10]:
def actors_with_largest_movie_gap_year(data):
    # Split the 'Actors' column and explode into individual rows
    actors_data = data.assign(Actors=data["Actors"].str.split('|')).explode("Actors")
    # Group by actor and find the min. and max. year for each actor
    # agg(): Aggregation functions to the grouped data
    actor_year_gap = actors_data.groupby("Actors").agg(min_year=("Year", "min"), max_year=("Year", "max"))
    # Calc. the gap between the minimum and maximum year
    actor_year_gap["gap"] = actor_year_gap["max_year"] - actor_year_gap["min_year"]
    # Find all of the actors with the largest gap
    max_gap = actor_year_gap["gap"].max()
    answer = actor_year_gap[actor_year_gap["gap"] == max_gap]
    # Count the number of rows
    answer_number_actors = len(answer)

    return answer, answer_number_actors

question_6, question_6_number_actors = actors_with_largest_movie_gap_year(data)
print(f"The total number of actors with largest maximum gap of years is {question_6_number_actors}")
question_6

The total number of actors with largest maximum gap of years is 53


,min_year,max_year,gap
Actors,,,
Abbie Cornish,2006,2016,10
Anne Hathaway,2006,2016,10
Audrey Tautou,2006,2016,10
Ben Kingsley,2006,2016,10
Ben Whishaw,2006,2016,10
Bob Balaban,2006,2016,10
Brad Pitt,2006,2016,10
Bryce Dallas Howard,2006,2016,10
Chiwetel Ejiofor,2006,2016,10


QUESTION 7

In [11]:
def find_all_actors_collaborate_with_JohnnyDepp_directly_indirectly(data):
    # Split the 'Actors' column into individual actors and explode into rows
    actors_data = data.assign(Actors=data["Actors"].str.split('|')).explode("Actors")
    
    # Create an adjacency list for collaborations (movie title -> set of actors)
    # Like "Movie 1": {"Johnny Depp", "Leonardo DiCaprio"}
    collaborations = actors_data.groupby("Title")["Actors"].apply(set).to_dict()
    
    # Using BFS to find all actors collaborating with Johnny Depp directly and indirectly
    answer = set()  # To keep track of actors we've already seen # Visited
    queue = ['Johnny Depp']  # Start the search with Johnny Depp
    
    while queue:
        current_actor = queue.pop(0)  
        if current_actor not in answer:
            answer.add(current_actor)  # Mark the actor as visited
            # Find all movies that current_actor has worked on and record the collaborated actors
            for movie_actors in collaborations.values():
                if current_actor in movie_actors:
                    queue.extend(movie_actors - answer)  # Add all unvisited co-actors to the queue
    
    # Remove Johnny Depp himself from the result
    answer.discard('Johnny Depp')
    
    return answer, len(answer)

question_7, question_7_number_collaborated_actors = find_all_actors_collaborate_with_JohnnyDepp_directly_indirectly(data)
print(f"The total number of actors that collaborate with Johnny Depp in direct and indirect ways is {question_7_number_collaborated_actors}")
question_7

The total number of actors that collaborate with Johnny Depp in direct and indirect ways is 1574


{'Sonoya Mizuno',
 'Benicio Del Toro',
 'Helen Mirren',
 'Jacob Tremblay',
 'Karen Moskow',
 'Will Dalton',
 'Megan Fox',
 'Tony Revolori',
 'Matthew Morrison',
 'Margot Robbie',
 'Rasmus Hardiker',
 'Shia LaBeouf',
 'Kevin Kline',
 'Julianna Margulies',
 'Jason Segel',
 'Jennifer Aniston',
 'Thomas M. Wright',
 'Sylvester Stallone',
 'Brandon T. Jackson',
 'Salim Kechiouche',
 "Jack O'Connell",
 'Jacki Weaver',
 'Keri Russell',
 'Robert Knepper',
 'Travis Tope',
 'Brittany Snow',
 'Jack McBrayer',
 'Nicolas Cage',
 'Damaine Radcliff',
 'Sean Faris',
 'Ving Rhames',
 'Michael Parks',
 'Kevin Costner',
 'Vincent Kartheiser',
 'Anthony Anderson',
 'Alec Baldwin',
 'Sheila Kelley',
 'Andrew Scott',
 "Fionn O'Shea",
 'Mel Gibson',
 'Mateo Arias',
 'Kelsey Grammer',
 'Terry Crews',
 'Michael B. Jordan',
 'Kati Outinen',
 'Zoe Bell',
 'Dakota Goyo',
 'Evan Rachel Wood',
 'Joe Cobden',
 'Elena Anaya',
 'Steven Yeun',
 'Mark Chao',
 'Camille Cottin',
 'Vicky Krieps',
 'Jorma Taccone',
 'Paul N